## 使用字符RNN生成莎士比亚文本

在2015年的一篇著名博客文章 "https://karpathy.github.io/2015/05/21/rnn-effectiveness/" 中，Andrej Karpathy展示了如何训练循环神经网络来预测句子中的下一个字符。这个char-RNN可以用来生成小说文本，每次一个字符。

In [2]:
# tf.keras.utils.get_file()函数下载莎士比亚的所有作品
import tensorflow as tf

shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [4]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [12]:
# 使用tf.keras.layers.TextVectorization层对此文本进行编码。我们将设置split="character"，基于字符进行编码，而不是默认的基于单词进行编码，并使用standardize="lower"将文本转换为小写（这将简化任务）：
text_vec_layer = tf.keras.layers.TextVectorization(split="character", standardize="lower")

text_vec_layer.adapt([shakespeare_text])


encoded = text_vec_layer([shakespeare_text])[0]
encoded


<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([21,  7, 10, ..., 22, 28, 12], dtype=int64)>

In [13]:
text_vec_layer.get_vocabulary()

['',
 '[UNK]',
 ' ',
 'e',
 't',
 'o',
 'a',
 'i',
 'h',
 's',
 'r',
 'n',
 '\n',
 'l',
 'd',
 'u',
 'm',
 'y',
 'w',
 ',',
 'c',
 'f',
 'g',
 'b',
 'p',
 ':',
 'k',
 'v',
 '.',
 "'",
 ';',
 '?',
 '!',
 '-',
 'j',
 'q',
 'x',
 'z',
 '3',
 '&',
 '$']

In [14]:
# 每个字符现在都映射到一个整数，从2开始。TextVectorization层将值0保留为填充词元(token)，并将值1保留为未知字符。
# 现在不需要这两个词元，因此从字符ID数中减去2，并计算不同字符的总数和总字符数：
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)

print(n_tokens)

39


将这个非常长的序列转换成一个窗口数据集，然后使用它来训练一个序列到序列的循环神经网络。目标序列将类似于输入序列，但是会向“未来”移动一个时间步。例如，数据集中的样本可能是一个由表示文本“to be or not to b”（不包括最后一个“e”）的字符ID组成的序列，相应的目标序列是一个由表示文本“o be or not to be”（包括最后一个“e”，但不包括开头的“t”）的字符ID组成的序列

In [9]:
# 将字符ID序列转换成输入/目标窗口对的数据集
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)

    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))


    if shuffle:
        ds = ds.shuffle(buffer_size=100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

- 它将序列作为输入（即编码文本），并创建一个包含所需长度的所有窗口的数据集。
- 它将长度增加1，因为我们需要将下一个字符放在目标序列中。
- 然后，它对窗口进行乱序处理（可选），将它们分批，将它们拆分为输入/输出对，并激活预取功能。

长度为11的窗口和批量大小3。每个窗口的起始索引都显示在旁边：

![准备乱序窗口的数据集](./images/RNN/p7.png)

In [16]:
# 大约90%的文本进行训练，5%用于验证，5%用于测试
length = 100  # length决定循环神经网络能学习的最长模式
tf.random.set_seed(42)

train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1_000_000: 1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

In [17]:
for data in valid_set.take(1):
    print(data)

(<tf.Tensor: shape=(32, 100), dtype=int64, numpy=
array([[ 5,  7,  0, ...,  6,  1,  0],
       [ 7,  0,  8, ...,  1,  0, 18],
       [ 0,  8,  1, ...,  0, 18,  6],
       ...,
       [ 4,  2,  0, ...,  3, 26, 10],
       [ 2,  0,  7, ..., 26, 10, 10],
       [ 0,  7,  6, ..., 10, 10,  2]], dtype=int64)>, <tf.Tensor: shape=(32, 100), dtype=int64, numpy=
array([[ 7,  0,  8, ...,  1,  0, 18],
       [ 0,  8,  1, ...,  0, 18,  6],
       [ 8,  1,  4, ..., 18,  6,  3],
       ...,
       [ 2,  0,  7, ..., 26, 10, 10],
       [ 0,  7,  6, ..., 10, 10,  2],
       [ 7,  6,  1, ..., 10,  2,  8]], dtype=int64)>)


### 构建和训练char-RNN模型

- 使用Embedding层作为第一层，以编码字符ID。Embedding层的输入维数是不同字符ID的数量，输出维数是可以调整的超参数——现在将其设置为16。Embedding层的输入将是形状为［批量大小，窗口长度］的二维张量，Embedding层的输出将是形状为［批量大小，窗口长度，嵌入大小］的三维张量。
- 为输出层使用Dense层：它必须具有39个单元(n_tokens)，因为文本中有39个不同的字符，希望在每个时间步输出每个可能字符的概率。每个时间步39个输出概率的总和应该为1，因此对Dense层的输出应用softmax激活函数。
- 最后，使用"sparse_categorical_crossentropy"损失和Nadam优化器编译此模型，并使用ModelCheckpoint回调函数在训练过程中保存最佳模型（以验证精度为标准）进行多个轮次的训练。

In [18]:
# 不用简单RNN,用带有长短记忆的GRU
#
# [[ 5,  7,  0, ...,  6,  1,  0]]  1 * 100
# [[ v5  v7  v0  .... v6  v1  v0]] 1 * 100 * 16

# n_tokens:39
# embedding： 39 * n 列的矩阵：  5行：  [w0, w1, ... wn-1]

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model", monitor="val_accuracy", save_best_only=True
)
history = model.fit(train_set, validation_data=valid_set, epochs=10, callbacks=[model_ckpt])

In [20]:
# 训练后搭建最终模型
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X-2), # 不使用填充词元(0)  和 未知词元 (1)
    model
])

In [23]:
y_proba = shakespeare_model.predict(["To be or not to be"])[0, -1]
y_proba
y_pred = tf.argmax(y_proba)  # 选择概率最高的字母ID

print(text_vec_layer.get_vocabulary())

text_vec_layer.get_vocabulary()[y_pred + 2]

1/1 [==============================] - 0s 41ms/step
['', '[UNK]', ' ', 'e', 't', 'o', 'a', 'i', 'h', 's', 'r', 'n', '\n', 'l', 'd', 'u', 'm', 'y', 'w', ',', 'c', 'f', 'g', 'b', 'p', ':', 'k', 'v', '.', "'", ';', '?', '!', '-', 'j', 'q', 'x', 'z', '3', '&', '$']


'l'

### 生成莎士比亚文本

要使用char-RNN模型生成新文本，可以将一些文本输入模型，让模型预测最有可能的下一个字母，将其添加到文本的末尾，然后将扩展后的文本输入模型来猜测下一个字母，以此类推。这叫作贪婪解码。但是在实践中，这往往导致相同的单词一遍又一遍地重复。

相反，可以使用TensorFlow的tf.random.categorical()函数随机采样下一个字符，采样概率等于估计概率。这将生成更多样化和有趣的文本。categorical()函数在给定类别对数概率(logits)的情况下，对随机类别指数进行采样。

In [38]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])
tf.random.set_seed(42)

n = 2000
tf.random.categorical(log_probas, num_samples=n)



<tf.Tensor: shape=(1, 2000), dtype=int64, numpy=array([[0, 1, 0, ..., 1, 0, 2]], dtype=int64)>

为了更好地控制生成文本的多样性，可以将logits（对数概率）除以一个称为“温度”的数字，这个数字可以根据需求进行调整。“温度”接近零将更偏向于高概率字符，而较高“温度”则会使所有字符获得相同的概率。通常在生成相对严谨和精确的文本（例如数学公式）时，较低的“温度”更为适用，而在生成更多样化且有创意的文本时，则适合用较高的“温度”。

In [39]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0,0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [8]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [41]:
tf.random.set_seed(42)
print(extend_text("To be or not to be", temperature=1))


1/1 [==============================] - 0s 34ms/step
To be or not to bedf3zwvcik :ua!&q. :phgr&;ubltcpzhp:'rv:cq3z!$ pau:


In [28]:
print(extend_text("To be or not to be", temperature=1))

1/1 [==============================] - 0s 16ms/step
To be or not to be good and wrong; but come. though which the fine.



In [29]:
print(extend_text("To be or not to be", temperature=100))

1/1 [==============================] - 0s 16ms/step
To be or not to bef ,mt'&ozfpady-$
wh!nse?pws3ert--vgerdjw?c-y-ewxnj


### 模拟生成莎士比亚文本的流程 生成名字

In [42]:
with open("datasets/dino.txt", "r") as f:
    dino_names = f.read()
    dino_names = dino_names.lower()

In [43]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character", standardize="lower")

text_vec_layer.adapt([dino_names])
encoded = text_vec_layer([dino_names])[0]
encoded

<tf.Tensor: shape=(19909,), dtype=int64, numpy=array([ 2,  2, 15, ...,  4,  4, 12], dtype=int64)>

In [44]:
text_vec_layer.get_vocabulary()

['',
 '[UNK]',
 'a',
 's',
 'u',
 'o',
 'r',
 '\n',
 'n',
 'i',
 'e',
 't',
 'l',
 'p',
 'h',
 'c',
 'g',
 'd',
 'm',
 'y',
 'b',
 'k',
 'v',
 'x',
 'z',
 'j',
 'w',
 'f',
 'q']

In [45]:
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)
n_tokens

27

In [46]:
end_of_name_encode = text_vec_layer(["\n"])[0,0].numpy() - 2
end_of_name_encode

5

In [62]:
# 把 Tensor 转成 numpy 数组
encoded_np = encoded.numpy()

# 存放 (X, Y) 对
names_X = []  # [第一个名字不包括换行符， 第二个名字不包括换行符， ...  最后一个名字不包括换行符]
names_Y = []  # [第一个名字有换行符，但无第一个字符， .....    , ...  最后一个名字有换行符，但无第一个字符]

# 临时缓冲区
current_name = []

for token in encoded_np:
    current_name.append(token)
    if token == end_of_name_encode:
        if len(current_name) > 1:  # 至少两个字符才能形成 X/Y
            X = current_name[:-1]
            Y = current_name[1:]
            names_X.append(X)
            names_Y.append(Y)
        current_name = []


current_name.append(end_of_name_encode)
if len(current_name) > 1:
    X = current_name[:-1]
    Y = current_name[1:]
    names_X.append(X)
    names_Y.append(Y)
    current_name = []

names_X[:5], names_Y[:5]
# 转成 TensorFlow 张量
X_dataset = tf.ragged.constant(names_X)
X_dataset
Y_dataset = tf.ragged.constant(names_Y)
Y_dataset

train_set = tf.data.Dataset.from_tensor_slices((X_dataset, Y_dataset)).shuffle(1000).batch(8)
print("共提取名字数:", len(names_X))


共提取名字数: 1536


In [63]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

# history = model.fit(X_dataset, Y_dataset, batch_size=1, epochs=10)
model.fit(train_set, epochs=20)

Epoch 1/20
192/192 [==============================] - 3s 4ms/step - loss: 2.4416 - accuracy: 0.3078
Epoch 2/20
192/192 [==============================] - 1s 4ms/step - loss: 1.9805 - accuracy: 0.4220
Epoch 3/20
192/192 [==============================] - 1s 5ms/step - loss: 1.8669 - accuracy: 0.4487
Epoch 4/20
192/192 [==============================] - 1s 4ms/step - loss: 1.7963 - accuracy: 0.4662
Epoch 5/20
192/192 [==============================] - 1s 4ms/step - loss: 1.7508 - accuracy: 0.4783
Epoch 6/20
192/192 [==============================] - 1s 4ms/step - loss: 1.7099 - accuracy: 0.4866
Epoch 7/20
192/192 [==============================] - 1s 4ms/step - loss: 1.6760 - accuracy: 0.4983
Epoch 8/20
192/192 [==============================] - 1s 4ms/step - loss: 1.6460 - accuracy: 0.5024
Epoch 9/20
192/192 [==============================] - 1s 4ms/step - loss: 1.6203 - accuracy: 0.5163
Epoch 10/20
192/192 [==============================] - 1s 4ms/step - loss: 1.5969 - accuracy: 0.5203

In [64]:
dino_name_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X -2),
    model
])

In [65]:
def my_next_char(model, text, temperature=1):
    y_proba = model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0,0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [66]:
def my_extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        char_gen_next = my_next_char(dino_name_model, text, temperature)
        if char_gen_next == "\n":
            break
        text += my_next_char(dino_name_model, text, temperature)
    return text

In [70]:
my_extend_text("xx")

1/1 [==============================] - 0s 33ms/step


'xxuwusaurus'

## 有状态的RNN

刚才的模型无法学习长度超100个字符的模式，使用有状态的RNN可以学习更长序列。

到目前为止，只使用了无状态RNN：在每次训练迭代中，模型从一个全是零的隐藏状态开始，然后在每个时间步更新这个状态，在最后一个时间步之后，将其丢弃，因为不再需要它了。

如果指示RNN在处理完一个训练批次后保留该最终状态，并将其用作下一个训练批次的初始状态，那么模型可以学习长期模式，尽管只通过短序列进行反向传播。这就是所谓的有状态RNN。

首先需要注意的是，只有在批次中的每个输入序列都从相应的上一批次序列结束的位置开始时才能建立有状态RNN。因此，构建有状态RNN时，首先需要使用非重叠顺序输入序列（而不是用于训练无状态RNN的随机重叠序列）。当创建tf.data.Dataset时，需要在调用window()方法时使用shift=length（而不是shift=1）。此外，不能调用shuffle()方法。

在为有状态RNN准备数据集时，批处理要比无状态RNN时更困难。如果调用batch(32)，那么32个连续的窗口将被放入同一个批次中，接下来的批次将无法继续从这些窗口的最后一个位置开始。第一个批次将包含窗口1到32，第二个批次将包含窗口33到64，因此如果考虑每个批次的第一个窗口（即窗口1和33），便会发现它们不是连续的。这个问题最简单的解决方案就是只使用批量大小1。下面的to_dataset_for_stateful_rnn()自定义实用函数使用这种策略来为有状态RNN准备数据集：

![为有状态RNN准备连续序列片段的数据集](./images/RNN/p10.png)

In [1]:
def to_dataset_for_stateful_rnn(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length+1, shift=length, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(length+1)).batch(1)

    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [3]:
import tensorflow as tf
text_vec_layer = tf.keras.layers.TextVectorization(split="character", standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() - 2
length = 100  # length决定循环神经网络能学习的最长模式

In [4]:
stateful_train_set = to_dataset_for_stateful_rnn(encoded[:1_000_000], length)
stateful_valid_set = to_dataset_for_stateful_rnn(encoded[1_000_000:1_060_000], length)
stateful_test_set = to_dataset_for_stateful_rnn(encoded[1_060_000:], length)

现在创建有状态RNN。当创建每个循环层时，需要将stateful参数设置True，因为有状态RNN需要知道批量大小（因为它将为批处理中的每个输入序列保留状态）。因此，必须在第一层设置batch_input_shape参数。请注意，可以不指定第二维度，因为输入序列可以具有任意长度

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16, batch_input_shape=[1, None]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

In [6]:
# 每个轮次结束时，需要在返回到文本开头之前重置状态
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [7]:
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model", monitor="val_accuracy", save_best_only=True
)
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(stateful_train_set, validation_data=stateful_valid_set, epochs=10, callbacks=[ResetStatesCallback(), model_ckpt])

Epoch 1/10
   9996/Unknown - 144s 14ms/step - loss: 1.8666 - accuracy: 0.4508INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


9999/9999 [==============================] - 147s 15ms/step - loss: 1.8665 - accuracy: 0.4508 - val_loss: 1.7065 - val_accuracy: 0.4911
Epoch 2/10
9995/9999 [============================>.] - ETA: 0s - loss: 1.5619 - accuracy: 0.5286INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


9999/9999 [==============================] - 139s 14ms/step - loss: 1.5618 - accuracy: 0.5287 - val_loss: 1.6223 - val_accuracy: 0.5136
Epoch 3/10
9996/9999 [============================>.] - ETA: 0s - loss: 1.4829 - accuracy: 0.5491INFO:tensorflow:Assets written to: my_shakespeare_model\assets


...

9999/9999 [==============================] - 142s 14ms/step - loss: 1.4037 - accuracy: 0.5694 - val_loss: 1.5435 - val_accuracy: 0.5397
Epoch 7/10
9999/9999 [==============================] - ETA: 0s - loss: 1.3918 - accuracy: 0.5723INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


9999/9999 [==============================] - 144s 14ms/step - loss: 1.3918 - accuracy: 0.5723 - val_loss: 1.5354 - val_accuracy: 0.5413
Epoch 8/10
9997/9999 [============================>.] - ETA: 0s - loss: 1.3825 - accuracy: 0.5746INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


9999/9999 [==============================] - 144s 14ms/step - loss: 1.3825 - accuracy: 0.5746 - val_loss: 1.5321 - val_accuracy: 0.5439
Epoch 9/10
9999/9999 [==============================] - 142s 14ms/step - loss: 1.3754 - accuracy: 0.5766 - val_loss: 1.5354 - val_accuracy: 0.5434
Epoch 10/10
9999/9999 [==============================] - 141s 14ms/step - loss: 1.3704 - accuracy: 0.5778 - val_loss: 1.5290 - val_accuracy: 0.5436


在训练后，只能使用与训练期间相同大小的批次进行预测。为避免此限制，请创建相同的无状态模型，并将有状态模型的权重复制到该模型

有趣的是，尽管char-RNN模型只是被训练来预测下一个字符，但是这个看似简单的任务实际上要求它学习一些更高级的任务。例如，查找“Great movie，I really”的下一个字符时，了解该句子是正面的是有帮助的，因此接下来的字符可能是“l”［表示“喜欢”(loved)］而不是“h”［表示“讨厌”(hated)］

OpenAI在一篇论文中描述了他们如何在大型数据集上训练大型char-RNN模型，并发现其中的一个神经元作为出色的情感分析分类器：尽管该模型在没有任何标签的情况下进行了训练，但“情感神经元”在情感分析基准测试中达到了最先进的性能。这预示并激发了NLP中无监督预训练的应用。

虽然批处理更难，但也不是不可能的。例如，可以将莎士比亚的文本分成32个长度相等的文本，为每个文本创建一个连续的输入序列数据集，最后使用tf.data.Dataset.zip(datasets).map(lambda*windows：tf.stack(windows))创建适当的连续批次，批次中的第n个输入序列恰好从上一个批次中第n个输入序列结束的地方开始

In [9]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0,0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [10]:
stateless_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])


stateless_model.build(tf.TensorShape([None, None]))
stateless_model.set_weights(model.get_weights())
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    stateless_model
])

tf.random.set_seed(42)
print(extend_text("to be or not to be", temperature=0.01))

1/1 [==============================] - 0s 16ms/step
to be or not to be a shall be a shall be a shall be a shall be a sha


In [11]:
# extra code – shows one way to prepare a batched dataset for a stateful RNN

import numpy as np

def to_non_overlapping_windows(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=length, drop_remainder=True)
    return ds.flat_map(lambda window: window.batch(length + 1))

def to_batched_dataset_for_stateful_rnn(sequence, length, batch_size=32):
    parts = np.array_split(sequence, batch_size)
    datasets = tuple(to_non_overlapping_windows(part, length) for part in parts)
    ds = tf.data.Dataset.zip(datasets).map(lambda *windows: tf.stack(windows))
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

list(to_batched_dataset_for_stateful_rnn(tf.range(20), length=3, batch_size=2))

[(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 0,  1,  2],
         [10, 11, 12]])>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 1,  2,  3],
         [11, 12, 13]])>),
 (<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 3,  4,  5],
         [13, 14, 15]])>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 4,  5,  6],
         [14, 15, 16]])>),
 (<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 6,  7,  8],
         [16, 17, 18]])>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 7,  8,  9],
         [17, 18, 19]])>)]

## 情感分析

生成文本的任务既有趣又有教学意义，但在真实项目中，NLP最常用的应用之一是文本分类——特别是情感分析。如果MNIST数据集上的图像分类是计算机视觉的“Hello World！”，那么IMDb电影评论数据集上的情感分析就是自然语言处理的“HelloWorld！”。IMDb数据集包括50000条英文电影评论（25000条用于训练，25000条用于测试），这些评论是从著名的互联网电影数据库 (https://imdb.com) 中提取的，每条评论还有一个简单的二元目标值，用以指示每条评论是负面的(0)还是正面的(1)。就像MNIST一样，IMDb电影评论数据集受欢迎是有原因的：它足够简单，可以在合理的时间内在CPU上处理。

In [19]:
import tensorflow_datasets as tfds

raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name="imdb_reviews",
    split=["train[:90%]", "train[90%:]", "test"],
    as_supervised=True
)
tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Administrator\tensorflow_datasets\imdb_reviews\plain_text\incomplete.LSDMPT_1.0.0\imdb_revi…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Administrator\tensorflow_datasets\imdb_reviews\plain_text\incomplete.LSDMPT_1.0.0\imdb_revi…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Administrator\tensorflow_datasets\imdb_reviews\plain_text\incomplete.LSDMPT_1.0.0\imdb_revi…

Dataset imdb_reviews downloaded and prepared to C:\Users\Administrator\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [20]:
for review, label in raw_train_set.take(4):
    print(review.numpy().decode("utf-8")[:200], "...")
    print("Label:", label.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0
Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Moun ...
Label: 0
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful perf ...
Label: 1


为了建立这个任务的模型，我们需要对文本进行预处理，但这一次我们将把文本拆分成单词而不是字符。为此，我们再次使用tf.keras.layers.TextVectorization层。请注意，它使用空格来确定单词边界，这在某些语言中不太合适。例如，汉语写作单词之间不使用空格，即使在英语中，空格也并不总是分词的最佳方式

幸运的是，有解决这些问题的方法。在2016年的一篇论文中探讨了几种子词级别的文本分词和重组方法。这样，即使模型遇到了它以前从未见过的生僻词，它仍然可以合理地猜测它的含义。例如，即使模型在训练期间从未见过单词smartest，如果它学到了单词smart并且还学到了后缀est的意思是“最”，它仍然可以推断出smartest的含义。作者评估的技术之一是字节对编码(Byte Pair Encoding，BPE)。BPE的工作原理是将整个训练集拆分为单个字符（包括空格），然后反复合并最常见的相邻字符对，直到词汇达到所需的大小。（从最基础的字符级词表开始，逐步地将出现频率最高的字符对（pair）合并，形成越来越长的子词序列。）